# COVID-19 India Daily updates
> Tracking coronavirus total cases, deaths and new cases in India on a daily basis

- author: Vaibhav Mahapatra

In [1]:
#hide

## Import essential packages below

import pandas as pd
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib as mpl
from IPython.core.display import display,HTML
%matplotlib inline

In [2]:
#hide
url = 'https://api.covid19india.org/csv/latest/state_wise_daily.csv'
df = pd.read_csv(url)
df.shape

(444, 41)

In [3]:
#hide
df.tail()

,Date,Status,TT,AN,AP,AR,AS,BR,CH,CT,...,PB,RJ,SK,TN,TG,TR,UP,UT,WB,UN
439,07-Aug-20,Recovered,50387,70,7594,81,1585,2445,43,231,...,381,1064,53,6488,1136,67,3432,304,2037,0
440,07-Aug-20,Deceased,937,3,89,0,6,12,3,10,...,22,10,0,119,12,5,63,10,52,0
441,08-Aug-20,Confirmed,65156,129,10080,68,2218,3992,52,447,...,998,1171,6,5883,2256,146,4660,501,2949,0
442,08-Aug-20,Recovered,52135,75,9151,104,1782,2408,52,263,...,820,968,11,5043,1091,224,2999,232,2064,0
443,08-Aug-20,Deceased,875,1,97,0,8,19,1,3,...,23,11,0,118,14,1,47,5,51,0


In [4]:
#hide
dt_today = df.iloc[-1][0]
dt_yday = df.iloc[-4][0]


In [5]:
#hide
df_today = df.groupby('Status').sum()
df_yday = df.iloc[:-4][:].groupby('Status').sum()

In [6]:
#hide
df_today.head()

,TT,AN,AP,AR,AS,BR,CH,CT,DN,DD,...,PB,RJ,SK,TN,TG,TR,UP,UT,WB,UN
Status,,,,,,,,,,,,,,,,,,,,,
Confirmed,2151928,1347,217040,2117,57613,75786,1426,11855,1512,0,...,22928,51328,873,290907,77513,6014,118038,9402,92615,0
Deceased,43464,20,1939,3,144,419,24,90,2,0,...,562,778,1,4808,615,37,2028,117,2005,0
Recovered,1479797,491,129615,1430,40592,48673,872,8582,1089,0,...,14860,37163,418,232618,54330,4084,69833,5963,65124,0


In [7]:
#hide
df_yday.head()

,TT,AN,AP,AR,AS,BR,CH,CT,DN,DD,...,PB,RJ,SK,TN,TG,TR,UP,UT,WB,UN
Status,,,,,,,,,,,,,,,,,,,,,
Confirmed,2086772,1218,206960,2049,55395,71794,1374,11408,1461,0,...,21930,50157,867,285024,75257,5868,113378,8901,89666,0
Deceased,41652,16,1753,3,130,388,20,77,2,0,...,517,757,1,4571,589,31,1918,102,1902,0
Recovered,1427662,416,120464,1326,38810,46265,820,8319,1038,0,...,14040,36195,407,227575,53239,3860,66834,5731,63060,0


In [8]:
#hide
summary = {
    'updated' : dt_today,
    'since' : dt_yday,
    'Cases' : df_today['TT'][0],
    'Deaths' : df_today['TT'][1],
    'PCases' : df_yday['TT'][0],
    'PDeaths' : df_yday['TT'][1],
    'Cases (+)' : df_today['TT'][0] - df_yday['TT'][0],
    'Deaths (+)' : df_today['TT'][1] - df_yday['TT'][1],
    }

summary

{'updated': '08-Aug-20',
 'since': '07-Aug-20',
 'Cases': 2151928,
 'Deaths': 43464,
 'PCases': 2086772,
 'PDeaths': 41652,
 'Cases (+)': 65156,
 'Deaths (+)': 1812}

In [9]:
#hide
overview = '''
<!-- #######  HTML!! #########-->
<h1 style="color: #5e9ca0; text-align: center;">India</h1>
<p style="text-align: center;">Last update: <strong>{update}</strong></p>
<p style="text-align: center;">Confirmed cases:</p>
<p style="text-align: center;font-size:24px;">{cases} (<span style="color: #ff0000;">+{new}</span>)</p>
<p style="text-align: center;">Confirmed deaths:</p>
<p style="text-align: center;font-size:24px;">{deaths} (<span style="color: #ff0000;">+{dnew}</span>)</p>
'''

In [10]:
#hide_input
# Fill the above HTML with the information from the summary dictionary

update = summary['updated']
cases = summary['Cases']
new = summary['Cases (+)']
deaths = summary['Deaths']
dnew = summary['Deaths (+)']

html = HTML(overview.format(update=update, cases=cases,new=new,deaths=deaths,dnew=dnew))

display(html)